In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import svm
import re
from sklearn.preprocessing import Imputer
from sklearn import model_selection
from numpy import random
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt 
import pickle
### Set path to the data set
dataset_path = "/home/ubuntu/Desktop/datapredict/77_cancer_proteomes_CPTAC_itraq.csv"
clinical_info = "/home/ubuntu/Desktop/datapredict/clinical_data_breast_cancer.csv"
pam50_proteins = "/home/ubuntu/Desktop/datapredict/PAM50_proteins.csv"

## Load data
data = pd.read_csv(dataset_path,header=0,index_col=0)
clinical = pd.read_csv(clinical_info,header=0,index_col=0)## holds clinical information about each patient/sample
pam50 = pd.read_csv(pam50_proteins,header=0)

## Drop unused information columns
data.drop(['gene_symbol','gene_name'],axis=1,inplace=True)

## Change the protein data sample names to a format matching the clinical data set
data.rename(columns=lambda x: "TCGA-%s" % (re.split('[_|-|.]',x)[0]) if bool(re.search("TCGA",x)) is True else x,inplace=True)

## Transpose data for the clustering algorithm since we want to divide patient samples, not proteins
data = data.transpose()

data_p50 = data.loc[:,data.columns.isin(pam50['RefSeqProteinID'])]
## Drop clinical entries for samples not in our protein data set
clinical = clinical.loc[[x for x in clinical.index.tolist() if x in data_p50.index],:]

## Add clinical meta data to our protein data set, note: all numerical features for analysis start with NP_ or XP_
merged = data.merge(clinical,left_index=True,right_index=True)

Prediction 1

In [3]:
x_train1 = merged.loc[:,[x for x in merged.columns if bool(re.search("Tumor|Status",x)) == True]]
y_train1 = merged.loc[:,[x for x in merged.columns if bool(re.search("Vital Status",x)) == True]]
x_train1.drop(['Tumor--T1 Coded','Vital Status'],axis=1,inplace=True)
x_train1 = x_train1.replace({'Negative' : 0,'Positive' : 1,'Equivocal':'NaN','T1' : 1, 'T2' : 2, 'T3' : 3, 'T4' : 4})
y_train1 = y_train1.replace({'DECEASED':0,'LIVING':1})

imputer = Imputer(missing_values='NaN', strategy='median', axis=1)
imputer = imputer.fit(x_train1)
x_train1 = imputer.transform(x_train1)
y_train1 = imputer.transform(y_train1)
y_train1 = y_train1.astype(int)

In [119]:
test_size = 0.9
seed = 1
X_train1, X_test1, Y_train1, Y_test1 = model_selection.train_test_split(x_train1, y_train1, test_size=test_size, random_state=seed)

In [120]:
model1 = svm.SVC(kernel='linear')
model1.fit(X_train1, np.ravel(Y_train1))
result1 = model1.score(X_test1, Y_test1)
print 'result 1 is:', result1

result 1 is: 0.916666666667


In [109]:
model2 = svm.SVC(kernel='poly')
model2.fit(X_train1, np.ravel(Y_train1))
result2 = model2.score(X_test1, Y_test1)
print 'result 2 is:', result2

result 2 is: 0.925


In [110]:
model3 = svm.SVC(kernel='sigmoid')
model3.fit(X_train1, np.ravel(Y_train1))
result3 = model3.score(X_test1, Y_test1)
print 'result 3 is:', result3

result 3 is: 0.925


In [111]:
model4 = svm.SVC()
model4.fit(X_train1, np.ravel(Y_train1))
result4 = model4.score(X_test1, Y_test1)
print 'result 4 is:', result4

result 4 is: 0.925


In [112]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train1, Y_train1)
clf.score(X_test1, Y_test1)

0.34999999999999998

In [113]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier() 
knn.fit(X_train1, Y_train1)
knn.score(X_test1, Y_test1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.92500000000000004

In [114]:
from sklearn import tree
model1 = tree.DecisionTreeClassifier()
model1.fit(X_train1, Y_train1)
model1.score(X_test1, Y_test1)

0.875